In [14]:
#export
"""
For operations that feel like the termination
"""
from collections import defaultdict
from typing import Iterator, Any
from k1lib.cli.init import BaseCli, Table
import torch, numbers; from k1lib import cli
__all__ = ["stdout", "file", "pretty", "display", "headOut", "intercept"]

In [15]:
#export
class stdout(BaseCli):
    """Prints out all lines. If not iterable, then print out the input raw"""
    def __ror__(self, it:Iterator[str]):
        try:
            it = iter(it)
            for line in it: print(line)
        except TypeError: print(it)
class file(BaseCli):
    def __init__(self, fileName:str):
        super().__init__(); self.fileName = fileName
    def __ror__(self, it:Iterator[str]) -> None:
        super().__ror__(it)
        with open(self.fileName, "w") as f:
            for line in it: f.write(f"{line}\n")

In [16]:
#export
class pretty(BaseCli):
    """Pretty prints a table"""
    def __ror__(self, it:Table[Any]) -> Iterator[str]:
        table = []; widths = defaultdict(lambda: 0)
        for row in it:
            _row = []
            for i, e in enumerate(row):
                e = f"{e}"; _row.append(e)
                widths[i] = max(len(e), widths[i])
            table.append(_row)
        for row in table:
            s = ""
            for w, e in zip(widths.values(), row):
                s += e.rstrip(" ").ljust(w+3)
            yield s

In [17]:
#export
def display(lines:int=10):
    """Convenience method for displaying a table"""
    f = pretty() | stdout()
    if lines is None: return f
    else: return cli.head(lines) | f
def headOut(lines:int=10):
    """Convenience method for head() | stdout()"""
    if lines is None: return stdout()
    else: return cli.head(lines) | stdout()

In [18]:
#export
class intercept(BaseCli):
    """Intercept flow at a particular point, analyze the object piped in, and
throw exception to stop flow. Example::

    3 | intercept()
"""
    def __ror__(self, s):
        print(type(s))
        if isinstance(s, (numbers.Number, str, bool)):
            print(s)
        elif isinstance(s, (tuple, list)):
            print(f"Length: {len(t)}")
            for e in s: print(f"- {type(e)}")
        elif isinstance(s, (np.ndarray, torch.Tensor)):
            print(f"Shape: {s.shape}")
        raise RuntimeError("intercepted")

In [20]:
try: 3 | intercept(); assert False
except: pass

<class 'int'>
3


In [21]:
!../../export.py cli/output

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.4
Uninstalling k1lib-0.4:
  Successfully uninstalled k1lib-0.4
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_baseClasses.py -> build/lib/k1lib
copying k1lib/